In [1]:
d = '../data/pre/' # raw data directory

import os
import pandas as pd
import numpy as np
from collections import Counter
from time import time
from imblearn.under_sampling import RandomUnderSampler


# ## Load all csv files into pandas DataFrames
'''
user = pd.read_csv('%suser.csv' % d)
ad = pd.read_csv('%sad.csv' % d)
position = pd.read_csv('%sposition.csv' % d)
test = pd.read_csv('%stest.csv' % d)
train = pd.read_csv('%strain.csv' % d)'''
ac = pd.read_csv('%sapp_categories.csv' % d)
ui = pd.read_csv('%suser_installedapps.csv' % d)
ua = pd.read_csv('%suser_app_actions.csv' % d)

In [115]:
tr_ori = pd.read_csv('../data/pre/new_generated_train.csv', index_col=0)
te_ori = pd.read_csv('../data/pre/new_generated_test.csv', index_col=0)

/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
ui_ids = ui.userID.unique()

In [4]:
ua_ids = ua.userID.unique()

In [5]:
len(ui_ids), len(ua_ids
                )

(1446105, 781112)

In [6]:
inters = set(ui_ids).intersection(ua_ids)

In [116]:
va = tr_ori.sample(frac=.1, random_state=3)
tr = tr_ori.drop(va.index, axis=0)

In [118]:
len(va), len(tr)

(374953, 3374575)

In [119]:
va_x = va.appCategory.values.reshape(-1,1)
va_y = va.label.values.reshape(-1,1)
tr_x = tr.appCategory.values.reshape(-1,1)
tr_y = tr.label.values.reshape(-1,1)

In [120]:
tr_ = tr.groupby('appCategory').apply(lambda df: np.mean(df.label))
va_ = va.groupby('appCategory').apply(lambda df: np.mean(df.label))

In [123]:
tr_,va_

(appCategory
 0      0.421053
 2      0.015507
 101    0.030436
 104    0.034925
 106    0.048780
 108    0.007061
 201    0.031483
 203    0.097034
 209    0.020315
 301    0.028169
 402    0.037585
 407    0.204886
 408    0.000000
 503    0.032605
 dtype: float64, appCategory
 0      0.444444
 2      0.015305
 101    0.030816
 104    0.034955
 106    0.000000
 108    0.005828
 201    0.031631
 203    0.092605
 209    0.020449
 301    0.026903
 402    0.037420
 407    0.193337
 408    0.000000
 503    0.026467
 dtype: float64)

In [11]:
ac_stat = tr.groupby('appCategory').apply(lambda df: len(df))

In [57]:
va_-np.ravel(model_.predict(tr_.index))

appCategory
0     -0.118241
2     -0.001088
101    0.021153
104    0.002107
106   -0.247342
108   -0.000698
201    0.000971
203    0.047513
209    0.000438
301    0.014674
402    0.008241
407    0.026503
408   -0.542655
503    0.020019
dtype: float64

In [75]:
model.predict(tr_.index)

array([[ 0.44356692],
       [ 0.02204597],
       [ 0.03755671],
       [ 0.03990374],
       [ 0.05462163],
       [ 0.00612368],
       [ 0.0381333 ],
       [ 0.11911406],
       [ 0.02481109],
       [ 0.03681537],
       [ 0.04339963],
       [ 0.20951852],
       [ 0.00422624],
       [ 0.54649371]], dtype=float32)

In [15]:
ac_stat.values
total_tr = 2*max(ac_stat.values)
new_x = []
new_y = []
for cate,occ in zip(ac_stat.index, ac_stat.values):
    tmp = tr.loc[tr.appCategory==cate]
    x = tmp.appCategory.values
    y = tmp.label.values
    x = list(x)
    y = list(y)
    x = x*(int(total_tr/len(x)))
    y = y*(int(total_tr/len(y)))    
    new_x += x
    new_y += y
new_x = np.array(new_x)
new_y = np.array(new_y)

In [16]:
new_y = new_y.reshape(-1,1)
new_x = new_x.reshape(-1,1)

In [17]:
new_y.shape

(47624892, 1)

In [30]:
ac_stat.index

Int64Index([0, 2, 101, 104, 106, 108, 201, 203, 209, 301, 402, 407, 408, 503], dtype='int64', name='appCategory')

In [14]:
tr_ac = tr.appCategory.values.reshape(-1,1)

In [15]:
tr_y = tr.label.values.reshape(-1, 1)

In [16]:
tr_ac_oh = pd.get_dummies(pd.concat([tr, te]).appCategory).values

In [17]:
tr_ac_oh = tr_ac_oh[:len(tr)]
tr_ac_oh.shape

(3749528, 14)

In [21]:
from keras.models import load_model, Sequential, Model
from keras.utils import plot_model, to_categorical
from keras.callbacks import TensorBoard, ModelCheckpoint, Callback
from keras.layers import Dense, Embedding, LSTM, GRU, SimpleRNN, BatchNormalization
from keras.layers import Dropout, Bidirectional, Flatten, Input, Reshape
from keras.layers.merge import Concatenate, Add, concatenate, add

Using TensorFlow backend.


In [74]:
# no ajust
np.random.seed(22)
i = Input(shape=(1,))
o = Embedding(np.max(tr_x)+1, 64)(i)
o = Flatten()(o)
o = Dense(64, activation='tanh')(o)
o = Dense(1, activation='sigmoid')(o)
model_ = Model(i,o)
model_.summary()
model_.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['binary_crossentropy'])
# print(model.layers[1].get_weights())
model_.fit(tr_x, tr_y,validation_data=(va_x,va_y), verbose=1, epochs=8, batch_size=256,  shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 1)                 0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 1, 64)             32256     
_________________________________________________________________
flatten_9 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 65        
Total params: 36,481
Trainable params: 36,481
Non-trainable params: 0
_________________________________________________________________
Train on 3374575 samples, validate on 374953 samples
Epoch 1/8
3374575/3374575 [==============================] - 20s - loss: 0.1132 - bin

In [109]:
np.random.seed(32)
i = Input(shape=(1,))
o = Embedding(np.max(new_x)+1, 64)(i)
o = Flatten()(o)
o = Dense(64, activation='tanh')(o)
o = Dense(1, activation='sigmoid')(o)
model = Model(i,o)
model.summary()
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['binary_crossentropy'])
# print(model.layers[1].get_weights())
model.fit(new_x, new_y,validation_data=(va_x,va_y), verbose=1, epochs=3, batch_size=4096,  shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        (None, 1)                 0         
_________________________________________________________________
embedding_25 (Embedding)     (None, 1, 64)             32256     
_________________________________________________________________
flatten_25 (Flatten)         (None, 64)                0         
_________________________________________________________________
dense_49 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_50 (Dense)             (None, 1)                 65        
Total params: 36,481
Trainable params: 36,481
Non-trainable params: 0
_________________________________________________________________
Train on 47624892 samples, validate on 374953 samples
Epoch 1/3
47624892/47624892 [==============================] - 69s - loss: 0.1983 - 

In [90]:
model.fit(new_x, new_y,validation_data=(va_x,va_y), verbose=1, epochs=1, batch_size=4086,  shuffle=True)

Train on 47624892 samples, validate on 374953 samples
Epoch 1/1
47624892/47624892 [==============================] - 69s - loss: 0.1976 - binary_crossentropy: 0.1976 - val_loss: 0.1135 - val_binary_crossentropy: 0.1135


In [91]:
va_df = va_.to_frame().reset_index()
dict_cat_prob = va_df.set_index(va_df.appCategory).to_dict()[0]
dict_cat_prob

{0: 0.5,
 2: 0.015029264214046822,
 101: 0.033932813030200203,
 104: 0.03612074382665767,
 106: 0.045454545454545456,
 108: 0.0070273074564920057,
 201: 0.031471307440693576,
 203: 0.095578673602080624,
 209: 0.020321163301600585,
 301: 0.028712234840368545,
 402: 0.039628654565608976,
 407: 0.2084440507824033,
 408: 0.0,
 503: 0.037386269644334162}

In [108]:
from sklearn.metrics import log_loss
# ideal_loss = log_loss(va_y, np.array([dict_cat_prob[c[0]] for c in va_x]).reshape(-1,1))
model_loss = log_loss(va_y, model.predict(va_x ))
model_loss_ = log_loss(va_y, model_.predict(va_x ))
'%.8f, %.8f, %.8f'%(ideal_loss-model_loss , model_loss_-model_loss,ideal_loss-model_loss_)

'-0.00006013, 0.00028786, -0.00034798'

In [86]:
np.random.seed(10)
i = Input(shape=(len(tr_ac_oh[0]),))
o = Dense(64, activation='tanh')(i)
o = Dense(1, activation='sigmoid')(o)
model_oh = Model(i,o)
model_oh.summary()
model_oh.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['binary_crossentropy'])
model_oh.fit(tr_ac_oh, tr_y, validation_split=.1, verbose=1, epochs=5, batch_size=2048,  shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 14)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 64)                960       
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 65        
Total params: 1,025
Trainable params: 1,025
Non-trainable params: 0
_________________________________________________________________
Train on 3374575 samples, validate on 374953 samples
Epoch 1/5
3374575/3374575 [==============================] - 5s - loss: 0.1263 - binary_crossentropy: 0.1263 - val_loss: 0.0981 - val_binary_crossentropy: 0.0981
Epoch 2/5
3374575/3374575 [==============================] - 5s - loss: 0.1142 - binary_crossentropy: 0.1142 - val_loss: 0.0971 - val_binary_crossentropy: 0.0971
Epoch 3/5
3374575/3374575 [

In [62]:
model.fit(tr_ac_oh, tr_y, validation_split=.1, verbose=1, epochs=5, batch_size=256,  shuffle=True)


Train on 3374575 samples, validate on 374953 samples
Epoch 1/5
3374575/3374575 [==============================] - 15s - loss: 0.1143 - binary_crossentropy: 0.1143 - val_loss: 0.0964 - val_binary_crossentropy: 0.0964
Epoch 2/5
3374575/3374575 [==============================] - 15s - loss: 0.1144 - binary_crossentropy: 0.1144 - val_loss: 0.0967 - val_binary_crossentropy: 0.0967
Epoch 3/5
3374575/3374575 [==============================] - 15s - loss: 0.1144 - binary_crossentropy: 0.1144 - val_loss: 0.0962 - val_binary_crossentropy: 0.0962
Epoch 4/5
3374575/3374575 [==============================] - 15s - loss: 0.1144 - binary_crossentropy: 0.1144 - val_loss: 0.0965 - val_binary_crossentropy: 0.0965
Epoch 5/5
3374575/3374575 [==============================] - 15s - loss: 0.1144 - binary_crossentropy: 0.1144 - val_loss: 0.0972 - val_binary_crossentropy: 0.0972


In [22]:
np.random.seed(10)
i = Input(shape=(1,))
o = Embedding(np.max(new_x)+1, 16)(i)
o = Flatten()(o)
o = Dense(64, activation='tanh')(o)
o = Dense(1, activation='sigmoid')(o)
model = Model(i,o)
model.summary()
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['binary_crossentropy'])
# print(model.layers[1].get_weights())
model.fit(new_x, new_y, validation_data=(va_x,va_y), verbose=1, epochs=2, batch_size=128,  shuffle=True)

NameError: name 'tr_ac' is not defined